In [ ]:
# source timesfm/bin/activate


# git clone https://github.com/google-research/timesfm.git
# cd timesfm
# pip install -e .

In [1]:
import sys
from pathlib import Path

for p in [Path.cwd(), *Path.cwd().parents]:
    if (p / "utils").exists():
        sys.path.insert(0, str(p))
        PROJECT_ROOT = p
        break

import numpy as np
import pandas as pd
import torch


torch.set_float32_matmul_precision("high")

/Users/jan/Documents/working papers/project 1/timesfm/lib/python3.13/site-packages/torch/__init__.py:1617: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'. Old settings, e.g, torch.backends.cuda.matmul.allow_tf32 = True, torch.backends.cudnn.allow_tf32 = True, allowTF32CuDNN() and allowTF32CuBLAS() will be deprecated after Pytorch 2.9. Please see https://pytorch.org/docs/main/notes/cuda.html#tensorfloat-32-tf32-on-ampere-and-later-devices (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/Context.cpp:85.)
  _C._set_float32_matmul_precision(precision)


In [2]:
from pathlib import Path

# I/O
CSV_PATH   = Path("/Users/jan/Documents/working papers/project 1/data/btc_1h_test.csv")
OUT_PRED_CSV = Path("timesfm25_zeroshot_btc_predictions.csv")

# Columns
TIME_COL   = "timestamp"
TARGET_COL = "close"

# Series frequency flag for your pipeline (TimesFM-2.5 itself does NOT require it)
FREQ_STR   = "1H"   # switch to "15T" when you swap to 15-minute data

# Forecasting
CONTEXT_LENGTH = 512    # configurable
HORIZON        = 1      # set to 1, 3, 5, 20, ...; predictions will have H columns
BATCH_SIZE     = 64     # batch windows for speed

In [3]:
df = pd.read_csv(CSV_PATH, parse_dates=[TIME_COL])
df = df[[TIME_COL, TARGET_COL]].dropna().sort_values(TIME_COL).reset_index(drop=True)

values = df[TARGET_COL].to_numpy(np.float32)
times  = df[TIME_COL].to_numpy()

# number of rolling forecast windows with full H-step targets available
n_windows = len(values) - CONTEXT_LENGTH - HORIZON + 1
assert n_windows > 0, "Not enough data for the chosen CONTEXT_LENGTH and HORIZON."

In [4]:
import timesfm

# Load checkpoint from HF; API per v2.5 model card
model = timesfm.TimesFM_2p5_200M_torch.from_pretrained("google/timesfm-2.5-200m-pytorch")

# Compile with point-forecast head only (disable quantiles for speed)
model.compile(
    timesfm.ForecastConfig(
        max_context=CONTEXT_LENGTH,
        max_horizon=HORIZON,
        normalize_inputs=True,
        use_continuous_quantile_head=False,  # point forecasts only
        force_flip_invariance=True,
        infer_is_positive=True,
        fix_quantile_crossing=True,
    )
)

AttributeError: module 'timesfm' has no attribute 'TimesFM_2p5_200M_torch'

In [ ]:
# Prepare arrays for results
pred_matrix = np.empty((n_windows, HORIZON), dtype=np.float32)

# We also store the 1-step ground truth for metrics, plus alignment times
y_true_1 = np.empty(n_windows, dtype=np.float32)
t_ref    = times[CONTEXT_LENGTH-1 : CONTEXT_LENGTH-1 + n_windows]          # last observed time in each context
t_tgt1   = times[CONTEXT_LENGTH     : CONTEXT_LENGTH     + n_windows]      # time of the first target (t+1)

# Batched rolling windows
start_indices = np.arange(n_windows, dtype=np.int64)
for s in range(0, n_windows, BATCH_SIZE):
    e = min(s + BATCH_SIZE, n_windows)
    idx = start_indices[s:e]

    # Build batch of contexts (each length = CONTEXT_LENGTH)
    batch_ctx = [values[i : i + CONTEXT_LENGTH] for i in idx]

    # Forecast exactly HORIZON steps; v2.5 returns (point, quantile) tuple
    point_forecast, _ = model.forecast(horizon=HORIZON, inputs=batch_ctx)
    point_forecast = np.asarray(point_forecast, dtype=np.float32)

    pred_matrix[s:e, :] = point_forecast

# Fill ground truth for the first step (t+1)
y_true_1[:] = values[CONTEXT_LENGTH : CONTEXT_LENGTH + n_windows]

# Assemble output DataFrame
pred_df = pd.DataFrame({
    "ref_timestamp": t_ref,             # context end (t)
    "timestamp": t_tgt1,                # first target time (t+1)
    "y_true": y_true_1,
    "y_pred": pred_matrix[:, 0],        # first-step prediction (convenience column for metrics)
})

# Also persist the full H-step vectors as separate columns
for h in range(HORIZON):
    pred_df[f"y_pred_h{h+1}"] = pred_matrix[:, h]

# Save
pred_df.to_csv(OUT_PRED_CSV, index=False)
pred_df.head()

In [ ]:
import pandas as pd
from utils.metrics import evaluate_forecast

dfm = pd.read_csv(OUT_PRED_CSV, parse_dates=["timestamp"])

res = evaluate_forecast(
    dfm["y_true"], dfm["y_pred"], dfm["timestamp"],
    starting_capital=100_000, threshold=0.005, fee_rate=0.001
)

# Minimal reporting
print(f"Final equity (no fees):   {float(res['equity_no_tc'].iloc[-1]):,.2f}")
print(f"Final equity (with fees): {float(res['equity_tc'].iloc[-1]):,.2f}")
print(f"Total fees paid:          {float(res['cum_fee'].iloc[-1]):,.2f}")
for k in ["arc", "asd", "mdd", "ir_star", "ir_starstar"]:
    v = res[k]
    print(f"{k}: {v:.6f}" if isinstance(v, float) else f"{k}: {v}")